In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import os

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

from datetime import date
import holidays
us_holidays = holidays.UnitedStates()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

In [59]:
df = pd.read_feather('Data/train_joined_processed_processed.feather')

current_vars = ['building_id', 'meter',  'air_temperature', 'cloud_coverage',
        'precip_depth_1_hr', 'wind_direction', 'month', 'day',
       'iso_weekend', 'hour', 'holidays', 'wind_speed']

# add wind_speed

train_df = df.sample(frac=0.8,random_state=11)
test_df = df.drop(train_df.index)

X_train = train_df[current_vars]
y_train = train_df.iloc[:,0]

X_test = test_df[current_vars]
y_test = test_df.iloc[:,0]

del df
del train_df
del test_df

In [60]:
cat_variables = ['meter', 'site_id', 'primary_use', 
                'iso_weekend',  'holidays',
                'wind_direction cat', 'precipitation', 'precipitation sign']

cat_variables_loc = list()
for col in cat_variables:
    if col in X_train.columns:
        cat_variables_loc.append(X_train.columns.get_loc(col))

In [57]:
import random
# samples = random.sample(set(building_list), 5)
# print(samples)
samples = [1058, 797, 1246, 889, 1190]

# building_list = X_test['building_id'].drop_duplicates()
# len(building_list)

In [61]:
%%time
y_train_list = pd.Series()
y_test_list = pd.Series()

prediction_train = pd.Series()
prediction_test = pd.Series()

for sample in samples: 
    X_train_b = X_train[X_train['building_id']==sample]
    y_train_b = y_train[X_train_b.index]
    X_test_b = X_test[X_test['building_id']==sample]
    y_test_b = y_test[X_test_b.index]
        
    bst = lgb.LGBMRegressor(n_estimators = 1000)
    bst.fit(X_train_b.values, y_train_b.values, categorical_feature=cat_variables_loc)
    
    prediction_on_X_train = bst.predict(X_train_b)
    prediction_on_X_train = pd.Series(prediction_on_X_train)
    prediction_on_X_train[prediction_on_X_train<0] = 0

    prediction_on_X_test = bst.predict(X_test_b)
    prediction_on_X_test = pd.Series(prediction_on_X_test)
    prediction_on_X_test[prediction_on_X_test<0] = 0
    
    y_train_list = pd.concat([y_train_list, y_train_b])
    y_test_list = pd.concat([y_test_list, y_test_b])

    prediction_train = pd.concat([prediction_train, prediction_on_X_train])
    prediction_test = pd.concat([prediction_test, prediction_on_X_test])

print('train: ', np.sqrt(mean_squared_log_error( y_train_list, prediction_train)))
print('test: ', np.sqrt(mean_squared_log_error( y_test_list, prediction_test)))

# no sea_level
# train:  1.0829716207630538
# test:  1.2138646566281732

# no 'site_id', 'primary_use', 'year_built', 'floor_count'
# train:  1.079449378290488
# test:  1.2049549023052715

# w dew_temperature: no good


# no 'cloud_coverage',
# train:  1.0779989566819095
# test:  1.1965857063109497

# no direction cat2 and cat
# train:  1.0780630545346153
# test:  1.1960939868889247


train:  1.0780630545346153
test:  1.1960939868889247
Wall time: 10.2 s
